# Continuous Systolic and Diastolic Blood Pressure Estimation Utilizing Long Short-term Memory Network

This Code is the implementation from papers [Continuous Systolic and Diastolic Blood Pressure Estimation Utilizing Long Short-term Memory Network](https://ieeexplore.ieee.org/document/8037207)
Using ECG and PPG signals to predict Systolic and Diastolic Blood Pressure

## Model Performance Evaluation
Using k-fold cross validation

## Step 1: Load Dataset
If you haven't run preprocess.py yet, please run the preprocess.py first otherwise this code below will not work.

In [25]:
import matplotlib.pyplot as plt
import numpy as np
from util import *
from model import *
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from keras.callbacks import EarlyStopping
import pandas as pd
from pyCompare import blandAltman
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import shuffle
import random

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
random.seed(0)

In [3]:
data_path = 'data/'

In [4]:
X_main = np.load(data_path+'x_main.npy')
y_main = np.load(data_path+'y_main.npy')
X_test = np.load(data_path+'x_test.npy')
y_test = np.load(data_path+'y_test.npy')

load max/min data for normalization

In [5]:
min_max = np.load(data_path+'min_max.npy')
ppg_all_min, ppg_all_max = min_max[0]
ecg_all_min, ecg_all_max = min_max[1]
abp_all_min, abp_all_max = min_max[2]

## Step 2: load model and train
If you've already trained the model skip this step.

In [6]:
if not os.path.isdir('./weights/'):
    os.mkdir('./weights/')

In [7]:
batch_size = 512
epochs = 10
timewindow = 32

find min/max for training data

In [10]:
ppg_train_conv = convert_to_2d(X_main, 0)
ecg_train_conv = convert_to_2d(X_main, 1)
abp_train_conv = convert_to_2d(y_main)

In [13]:
ppg_train_min, ppg_train_max = find_max_min(ppg_train_conv)
ecg_train_min, ecg_train_max = find_max_min(ecg_train_conv)
abp_train_min, abp_train_max = find_max_min(abp_train_conv)

**Save min/max for training data to file**

In [14]:
np.save(data_path+'min_max_train.npy', np.array([
    [ppg_train_min, ppg_train_max],
    [ecg_train_min, ecg_train_max],
    [abp_train_min, abp_train_max]
]))

define k-fold and number of splits (normally we chose 5 or 10)

In [23]:
kf = KFold(n_splits=5, random_state=42)
count = 1
results = []

Each iteration performs:
- convert dataset to timeseries
- train the model and collect the performance

In [ ]:
for t, v in kf.split(X_main):
    print('='*30)
    print('Starting fold: ', count)
    print('-'*30)
    X_train = np.array([X_main[i] for i in t])
    y_train = np.array([y_main[i] for i in t])
    X_valid = np.array([X_main[i] for i in v])
    y_valid = np.array([y_main[i] for i in v])
    
    encoder_input = []
    decoder_output = []
    decoder_input = []
    print('Start converting training set to timeseries...')
    for i in range(X_train.shape[0]):
        s_x, z_y, s_y = to_timeseries(X_train[i], y_train[i], is_train=True, max_len=timewindow)
        encoder_input.extend(s_x)
        decoder_output.extend(s_y)
        decoder_input.extend(z_y)
    encoder_input = np.array(encoder_input)
    decoder_output = np.array(decoder_output)
    decoder_input = np.array(decoder_input)
    print('Done converting training set to timeseries...')
    print('Num of sequences: ', encoder_input.shape[0])
    print('-'*30)
    
    encoder_input_valid = []
    decoder_output_valid = []
    decoder_input_valid = []
    print('Start converting validation set to timeseries...')
    for i in range(X_valid.shape[0]):
        s_x, z_y, s_y = to_timeseries(X_valid[i], y_valid[i], is_train=False, max_len=timewindow)
        encoder_input_valid.extend(s_x)
        decoder_output_valid.extend(s_y)
        decoder_input_valid.extend(z_y)
    encoder_input_valid = np.array(encoder_input_valid)
    decoder_output_valid = np.array(decoder_output_valid)
    decoder_input_valid = np.array(decoder_input_valid)
    print('Done converting validation set to timeseries...')
    print('Num of sequences: ', encoder_input_valid.shape[0])
    print('='*30)
    
    model = create_model()
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=2)
    ]
    model.fit([encoder_input, decoder_input], decoder_output, 
              validation_data=([encoder_input_valid, decoder_input_valid], decoder_output_valid)
              batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1, callbacks=callbacks)
    
    result = model.evaluate([encoder_input_valid, decoder_input_valid], decoder_output_valid, batch_size=batch_size)
    results.append(result[0])
    
    count += 1

## Step 3: Experimental Results

In [ ]:
print('Averange RMSE result from this model: ', np.mean(results))